# Reviewer response formal tests global patterns

In this notebook we formally test whether the genetic associations are specific or not.

Inputs:
- z scores from mvGWAS from MOSTEST (because they take SE into account)
- CCA loadings from CCA with polygenic scores
- z scores (beta/se) from burden regression

We use an independent samples t-test to test differences between:
- Left intra-hemispheric connectivity vs. right intra-hemispheric connectivity
- Intra-hemispheric vs. interhemispheric connectivity

We use a one-sample t-test to test whether the overall pattern is significantly different from zero

We output to csv-files.

Last update: 2024-06-26, JS Amelink

In [1]:
import pandas as pd
import numpy as np
import os
import h5py
import re
from scipy.stats import ttest_ind, ttest_1samp

## Helper functions

In [2]:
def decide_pheno(brain_pheno):
    """
    Decides which phenotype category based on filename.
    Input:
    - brain pheno name
    Output:
    - category
    
    Categories:
    edges, edges HD
    """
        
        #split edge name
    fn_split=brain_pheno.split('*')
        
        #conditional for categorization:
    if re.search("-L", fn_split[0]) is not None and re.search("-L", fn_split[1]) is not None:
        sub = 'L'
    elif re.search("-R", fn_split[0]) is not None and re.search("-R", fn_split[1]) is not None:
        sub = 'R'
    elif re.search("-L", fn_split[0]) is not None and re.search("-R", fn_split[1]) is not None:
        sub = 'inter'
    elif re.search("-R", fn_split[0]) is not None and re.search("-L", fn_split[1]) is not None:
        sub = 'inter'
    else:
        sub = 'L-R'
                
    return sub

def add_row(df, new_data, column_names, index_name, np_array=True):
    if np_array:
        df_new = pd.DataFrame(data=new_data.reshape((1, len(column_names))), columns=column_names, index=[index_name])
    else:
        df_new = pd.DataFrame.from_dict(new_data)
    return pd.concat([df, df_new], axis=0)
        
def get_snp(chr_no, snp_id):
    """
    """
    if chr_no == 23:
        bim_fn = "/data/clusterfs/lag/users/jitame/SENT_CORE/geno/mostest/in/mostest_geno_in_cXY.bim"
    else:
        bim_fn = "/data/clusterfs/lag/users/jitame/SENT_CORE/geno/mostest/in/mostest_geno_in_c{}.bim".format(chr_no)
    
    bim = pd.read_csv(bim_fn, sep='\t', header=None, names='CHR SNP GP BP A1 A2'.split())
    snp = bim.SNP.values == snp_id
    return snp, bim.loc[bim.SNP.values == snp_id, :].to_dict()

def get_file_name_mat(cat, chr_no):
    if chr_no == 23:
        return "/data/clusterfs/lag/users/jitame/SENT_CORE/geno/mostest/out/sent_edges/edges_chrXY_zmat.mat"
    if cat == "edges":
        return "/data/clusterfs/lag/users/jitame/SENT_CORE/geno/mostest/out/sent_edges/edges_chr{}_zmat.mat".format(chr_no)
    if cat == "edges_HD":
        return "/data/clusterfs/lag/users/jitame/SENT_CORE/geno/mostest/out/sent_edges_asym/edges_asym_chr{}_zmat.mat".format(chr_no)

def get_betas_zs(fname, snps):
    with h5py.File(fname, 'r') as h5file:
        beta_orig = np.array(h5file['beta_orig'])
        zmat_orig=np.array(h5file['zmat_orig'])
        #pval_orig = stats.norm.sf(np.abs(zmat_orig))*2.0

        betas_snp = beta_orig[:, snps]
        z_snp = zmat_orig[:, snps]
    
        return betas_snp, z_snp
    
def data_munger_MOSTEST_betas(snp_df, col_names, cat):
    """
    Load underlying beta's from MOSTEST analysis
    """
    
    beta_df = pd.DataFrame(columns=col_names)
    z_df = pd.DataFrame(columns=col_names)
    snp_info_df = pd.DataFrame(columns='CHR SNP GP BP A1 A2'.split())
    
    for snp_no in snp_df.index.values:
        snp, snp_all = get_snp(snp_df.chr_no[snp_no], snp_df.ID[snp_no])
    
        betas_snp, pval_snp = get_betas_zs(fname=get_file_name_mat(cat, snp_df.chr_no[snp_no]),
                                               snps=snp)
        
        beta_df = add_row(df = beta_df,
                          new_data = betas_snp,
                          column_names = col_names,
                          index_name = snp_df.ID[snp_no])
        
        z_df = add_row(df = z_df,
                          new_data = pval_snp,
                          column_names = col_names,
                          index_name = snp_df.ID[snp_no])
        
        snp_info_df = add_row(df = snp_info_df,
                              new_data = snp_all,
                              column_names = 'CHR SNP GP BP A1 A2'.split(),
                              index_name = snp_df.ID[snp_no],
                              np_array=False)
    
    return beta_df, z_df, snp_info_df

def load_column_names_first_line(fn):
    return [str(x) for x in open( fn ).read().split('\n')[0].split("\t") ]

def ttest_all(df):
    genos = list(df.columns)[:-1]
    alpha = 0.05 / ((len(df.columns)-1)*3)

    a = df[df["Category"] == "L"].iloc[:,:-1].to_numpy(dtype=np.float64)
    b = df[df["Category"] == "R"].iloc[:,:-1].to_numpy(dtype=np.float64)
    c = np.vstack([a, b])
    d = df[df["Category"] == "inter"].iloc[:,:-1].to_numpy(dtype=np.float64)
    e = np.vstack([c,d])

    mean_l = np.mean(a, axis=0)
    mean_r = np.mean(b, axis=0)
    mean_intra = np.mean(c, axis=0)
    mean_inter = np.mean(d, axis=0)
    mean_all = np.mean(e, axis=0)
    
    std_l = np.std(a, axis=0)
    std_r = np.std(b, axis=0)
    std_intra = np.std(c, axis=0)
    std_inter = np.std(d, axis=0)
    std_all = np.std(e, axis=0)
    
    #test different from 0
    global_t, global_p = ttest_1samp(e, popmean=0)
    
    #test differences
    lr_t, lr_p = ttest_ind(a, b)
    int_t, int_p = ttest_ind(c, d)
    
    #bonferroni correction
    bonf_global = global_p < alpha
    bonf_lr = lr_p < alpha
    bonf_int = int_p < alpha
    
    log10p_global = -np.log10(global_p)
    log10p_lr = -np.log10(lr_p)
    log10p_int = -np.log10(int_p)
    
    t_df = pd.DataFrame([mean_all, std_all, global_t, log10p_global, bonf_global, mean_l, std_l, mean_r, std_r, lr_t, log10p_lr, bonf_lr, mean_intra, std_intra, mean_inter, std_inter, int_t, log10p_int, bonf_int]).T
    t_df.index = genos
    t_df.columns = ["Mean_all", "Std_all", "T_test_1samp_all", "Log10P_all", "Significance_all", "Mean_L", "Std_L", "Mean_R", "Std_R", "T_test_LR", "Log10P_LR", "Signficant_LR", "Mean_intra", 'Std_intra', "Mean_inter", "Std_inter", "T_test_intra_inter", "Log10P_intra_inter", "Significant_intra_inter"]

    return t_df

def ttest_asym(df):
    alpha = 0.05/df.shape[1]
    print("Alpha: {}".format(alpha))
    genos = list(df.columns)

    z = df.to_numpy(dtype=np.float64)

    mean_z = np.mean(z, axis=0)
    std_z = np.std(z, axis=0)
    t, p = ttest_1samp(z, popmean=0)

    log10p = -np.log10(p)
    
    sig = p < alpha
    print(p)

    t_df = pd.DataFrame([mean_z, std_z, t, log10p, sig]).T
    t_df.index = genos
    t_df.columns = ["Mean_all", "Std_all", "T_test_1samp_all", "Log10P_all", "Significance"]
    
    return t_df 

def add_gene_names(data):
    """
    ADDS GENE NAME BASED ON ENSEMBL ID
    """
    
    from pyensembl import EnsemblRelease
    data_ens = EnsemblRelease(108)
    
    #parse gene ids and names
    data["gene_id"] = [id_name.split(".")[0] for id_name in data["ID"]]
    data["gene_name"] = [get_gene_name(gene_id, data_ens) for gene_id in data["gene_id"]]
    
    return data

def get_gene_name(gene_id, data_ens):
    """
    """
    try:
        gene_name = data_ens.gene_name_of_gene_id(gene_id)
    except ValueError:
        gene_name = ""
    return gene_name

def load_column_names(fn):
    return [str(x) for x in open( fn ).read().split('\n')[:-1]]

## Language network: SNPS + polygenic scores

In [3]:
#load snp data
common_all_lang = pd.DataFrame.from_dict({"chr_no":[2,3,3,3,3,4,10,10,11,14,17,17,18,23], "ID":["rs2717066", "rs17273020", "rs35124509", "rs143322006", "rs2279829", "rs2131466", "rs2274224", "rs4469802", "rs10897597", "rs468213", "rs4924967", "rs2732687", "rs7234875", "rs2360257"]})
col_names_lang = load_column_names_first_line("/data/clusterfs/lag/users/jitame/SENT_CORE/pheno/sent_edges_N29681_resid_mostest.txt")
beta_lang, z_lang, snp_lang = data_munger_MOSTEST_betas(common_all_lang, col_names=col_names_lang, cat="edges")

#load CCA data
cca_lang = pd.read_csv("/data/clusterfs/lag/users/jitame/SENT_CORE/geno/CCA_loadings.csv", index_col=0)

In [4]:
#set up data nicely
z_lang = z_lang.transpose()
cca_lang = cca_lang.transpose()

cca_lang.to_csv("/data/clusterfs/lag/users/jitame/SENT_CORE/geno/CCA_loadings_T.csv")

#classify edges
z_lang["Category"] = [decide_pheno(edge) for edge in z_lang.index.values]
cca_lang["Category"] = [decide_pheno(edge) for edge in cca_lang.index.values]

In [5]:
#get t-tests
snps_t = ttest_all(z_lang)
cca_t = ttest_all(cca_lang)

snps_t = snp_lang.set_index("SNP").join(snps_t)

cca_t.to_csv("/data/clusterfs/lag/users/jitame/SENT_CORE/geno/CCA_ttest.csv")
snps_t.to_csv("/data/clusterfs/lag/users/jitame/SENT_CORE/geno/snps_lang_ttest.csv")

In [6]:
cca_t.head()

,Mean_all,Std_all,T_test_1samp_all,Log10P_all,Significance_all,Mean_L,Std_L,Mean_R,Std_R,T_test_LR,Log10P_LR,Signficant_LR,Mean_intra,Std_intra,Mean_inter,Std_inter,T_test_intra_inter,Log10P_intra_inter,Significant_intra_inter
score_read,0.052587,0.147355,8.943215,17.376424,True,0.155013,0.13285,0.042903,0.120708,7.700279,12.715747,True,0.098958,0.138751,0.008657,0.141699,8.056898,14.402327,True
score_dyslexia,0.031073,0.131207,5.934905,8.313121,True,-0.025714,0.139464,0.008606,0.120059,-2.299279,1.654296,False,-0.008554,0.131251,0.068615,0.119613,-7.701175,13.277553,True
score_hand,0.068341,0.116232,14.734495,41.694742,True,0.008906,0.098009,0.050331,0.10976,-3.470828,3.226225,True,0.029618,0.106092,0.105025,0.113487,-8.583301,16.139163,True


In [7]:
snps_t.head(n=14)

,CHR,GP,BP,A1,A2,Mean_all,Std_all,T_test_1samp_all,Log10P_all,Significance_all,...,T_test_LR,Log10P_LR,Signficant_LR,Mean_intra,Std_intra,Mean_inter,Std_inter,T_test_intra_inter,Log10P_intra_inter,Significant_intra_inter
SNP,,,,,,,,,,,,,,,,,,,,,
rs2717066,2,0,58029645,A,C,0.372232,1.454328,6.414026,9.554491,True,...,1.953681,1.28688,False,0.35788,1.624558,0.385828,1.272069,-0.240525,0.091514,False
rs17273020,3,0,17415705,T,C,1.304267,1.279956,25.535918,98.464863,True,...,-6.577837,9.681313,True,1.265073,1.393434,1.341399,1.161054,-0.746646,0.341457,False
rs35124509,3,0,89521693,C,T,-0.734473,2.758324,-6.672832,10.258363,True,...,-0.91558,0.442959,False,-0.920123,2.947472,-0.558593,2.553885,-1.643908,0.996986,False
rs143322006,3,0,134976392,T,C,-1.077874,1.269232,-21.281713,75.360026,True,...,-4.332051,4.696946,True,-0.948407,1.326237,-1.200527,1.199942,2.498403,1.89516,False
rs2279829,3,0,147106319,T,C,1.022422,1.754194,14.606045,41.082441,True,...,1.311428,0.719643,False,1.209024,1.96129,0.845642,1.511288,2.606577,2.028602,False
rs2131466,4,0,28817206,G,T,-0.277558,1.65689,-4.197969,4.511132,True,...,-3.410432,3.132958,True,-0.201433,1.739871,-0.349676,1.570848,1.120878,0.580426,False
rs2274224,10,0,96039597,C,G,1.925297,1.829976,26.365271,102.981673,True,...,0.686626,0.307292,False,2.264132,1.778745,1.604297,1.819932,4.587804,5.266702,True
rs4469802,10,0,134309194,A,G,1.130344,1.749955,16.186901,48.784863,True,...,4.136706,4.340585,True,1.451991,1.762838,0.825627,1.681846,4.553116,5.197138,True
rs10897597,11,0,80057231,A,G,-0.73096,1.208722,-15.154677,43.715268,True,...,0.241499,0.091874,False,-0.893585,1.274368,-0.576894,1.121514,-3.307585,3.002246,True


## Hemispheric differences: SNPS + polygenic scores

In [8]:
common_top_asym = pd.DataFrame.from_dict({"chr_no":[3,3,3], "ID":["rs7635916", "rs2279829", "rs13321297"]})
col_names_asym = load_column_names_first_line("/data/clusterfs/lag/users/jitame/SENT_CORE/pheno/sent_edges_asym_N29681_resid_mostest.txt")
beta_asym, z_asym, snp_asym = data_munger_MOSTEST_betas(common_top_asym, col_names=col_names_asym, cat="edges_HD")

cca_asym = pd.read_csv("/data/clusterfs/lag/users/jitame/SENT_CORE/geno/CCA_loadings_asym.csv", index_col=0)

In [9]:
#turn data
z_asym = z_asym.transpose()
cca_asym = cca_asym.transpose()

#do t-testing
t_asym = ttest_asym(z_asym)
t_asym = snp_asym.set_index("SNP").join(t_asym)
CCA_t_asym = ttest_asym(cca_asym)

#save
t_asym.to_csv("/data/clusterfs/lag/users/jitame/SENT_CORE/geno/snps_t_asym.csv")
CCA_t_asym.to_csv("/data/clusterfs/lag/users/jitame/SENT_CORE/geno/cca_t_asym.csv")

t_asym.head()

Alpha: 0.016666666666666666
[5.16641539e-01 2.24640791e-01 4.31382524e-14]
Alpha: 0.016666666666666666
[1.01201159e-14 1.04991842e-05 1.03738157e-03]


,CHR,GP,BP,A1,A2,Mean_all,Std_all,T_test_1samp_all,Log10P_all,Significance
SNP,,,,,,,,,,
rs7635916,3,0,89633140,A,G,-0.111436,1.729337,-0.650796,0.286811,False
rs2279829,3,0,147106319,T,C,0.182701,1.510362,1.22169,0.648511,False
rs13321297,3,0,17528764,A,T,-1.084681,1.249534,-8.767063,13.365137,True


In [10]:
CCA_t_asym.head()

,Mean_all,Std_all,T_test_1samp_all,Log10P_all,Significance
score_read,0.16929,0.188859,9.053001,13.994815,True
score_dyslexia,-0.068732,0.149694,-4.637187,4.978844,True
score_hand,-0.053669,0.160493,-3.377257,2.984061,True


## Rare variant burden scores: language network + HDs

In [11]:
rare_genes = pd.read_csv(os.path.join("/data/clusterfs/lag/users/jitame/SENT_CORE/geno/regenie_gene_based/", "all_exome_results.csv"), engine="pyarrow")
rare_genes = add_gene_names(rare_genes)

data_burden_strict = rare_genes[np.array([rare_genes["TEST"] == "ADD", rare_genes["ALLELE1"] == "Strict.0.01"]).all(0).reshape((len(rare_genes), 1))]
data_burden_broad = rare_genes[np.array([rare_genes["TEST"] == "ADD", rare_genes["ALLELE1"] == "Broad.0.01"]).all(0).reshape((len(rare_genes), 1))]

In [12]:
genes=["MANEAL", "DUSP29", "SLC25A48", "TRIP11", "NIBAN1"]

heritable_lang = load_column_names("/data/clusterfs/lag/users/jitame/SENT_CORE/heritable_edges.txt")
heritable_b_lang = ["BETA_sent_edges_"+x for x in heritable_lang]
heritable_se_lang = ["SE_sent_edges_"+x for x in heritable_lang]
lang_burden_broad_sig = data_burden_broad.set_index("gene_name").loc[genes, :]

beta_df_broad = lang_burden_broad_sig[heritable_b_lang]
beta_df_broad.columns=[x[16:] for x in beta_df_broad.columns]

se_df_broad = lang_burden_broad_sig[heritable_se_lang]
se_df_broad.columns=[x[16:] for x in se_df_broad.columns]

z_np = beta_df_broad.to_numpy() / se_df_broad.to_numpy()

In [13]:
genes = ["WDCP", "DDX25"]

heritable_asym = load_column_names("/data/clusterfs/lag/users/jitame/SENT_CORE/heritable_edges_asym.txt")
heritable_b_asym = ["BETA_sent_edges_"+x for x in heritable_asym]
heritable_se_asym = ["SE_sent_edges_"+x for x in heritable_asym]
asym_strict_sig = data_burden_strict.set_index("gene_name").loc[genes, :]

beta_df_strict = asym_strict_sig[heritable_b_asym]
beta_df_strict.columns=[x[16:] for x in beta_df_strict.columns]

se_df_strict = asym_strict_sig[heritable_se_asym]
se_df_strict.columns=[x[16:] for x in se_df_strict.columns]

z_np_asym = beta_df_strict.to_numpy() / se_df_strict.to_numpy()

In [14]:
z_df = pd.DataFrame(z_np, columns=beta_df_broad.columns, index=beta_df_broad.index).T
z_df_asym = pd.DataFrame(z_np_asym, columns=beta_df_strict.columns, index=beta_df_strict.index).T

z_df["Category"] = [decide_pheno(edge) for edge in z_df.index.values]

In [15]:
rare_t = ttest_all(z_df)
rare_t_asym = ttest_asym(z_df_asym)

rare_t.to_csv("/data/clusterfs/lag/users/jitame/SENT_CORE/geno/rare_t_lang.csv")
rare_t_asym.to_csv("/data/clusterfs/lag/users/jitame/SENT_CORE/geno/rare_t_asym.csv")

Alpha: 0.025
[4.27917297e-05 8.45808673e-21]


In [16]:
rare_t.head()

,Mean_all,Std_all,T_test_1samp_all,Log10P_all,Significance_all,Mean_L,Std_L,Mean_R,Std_R,T_test_LR,Log10P_LR,Signficant_LR,Mean_intra,Std_intra,Mean_inter,Std_inter,T_test_intra_inter,Log10P_intra_inter,Significant_intra_inter
MANEAL,-1.113141,0.884362,-31.542788,130.867668,True,-1.144357,0.752854,-1.00325,0.995739,-1.393627,0.78397,False,-1.073803,0.885506,-1.150408,0.881659,1.085126,0.555514,False
DUSP29,-0.761958,1.039243,-18.373575,59.935635,True,-0.835673,0.890321,-0.793821,1.136983,-0.357308,0.141999,False,-0.814747,1.021342,-0.711948,1.053485,-1.239505,0.666306,False
SLC25A48,0.800083,1.08167,18.536168,60.783021,True,0.663567,1.156638,0.907603,1.097782,-1.886722,1.22076,False,0.785585,1.134176,0.813817,1.029271,-0.326684,0.128418,False
TRIP11,1.131913,0.971633,29.193812,118.306362,True,1.621236,1.022125,0.80931,0.877088,7.432189,11.961986,True,1.215273,1.035286,1.052941,0.900126,2.098303,1.440358,False
NIBAN1,0.818342,1.057588,19.390913,65.271162,True,0.400703,0.938554,1.407184,1.081924,-8.663581,15.560182,True,0.903943,1.130917,0.737246,0.976185,1.97884,1.316313,False


In [17]:
rare_t_asym.head()

,Mean_all,Std_all,T_test_1samp_all,Log10P_all,Significance
WDCP,-0.491599,1.160875,-4.276869,4.36864,True
DDX25,-1.54418,1.320657,-11.808861,20.072728,True


In [18]:
z_df.to_csv("/data/clusterfs/lag/users/jitame/SENT_CORE/geno/rare_z_lang.csv")
z_df_asym.to_csv("/data/clusterfs/lag/users/jitame/SENT_CORE/geno/rare_z_asym.csv")